- load pdf
- get image of each pdf
- convert page to image
- image input to llm
- get translations

In [7]:
from pdf2image import convert_from_path, pdfinfo_from_path
import requests
import base64
import os
from tqdm import tqdm
from ollama import chat
from pathlib import Path


In [8]:
PDF_PATH = "../data/Rigveda Hindi - Dr Ganga Sahay Sharma.pdf"
# OLLAMA_URL = "http://localhost:11434/api/generate"
MODEL_NAME = "gemma3:12b"
OUTPUT_FILE = "ollama_output.txt"
IMG_DIR = Path("../data/rigveda_images")  # Target folder



In [5]:
def encode_image(img_path):
    with open(img_path, "rb") as f:
        return base64.b64encode(f.read()).decode()

In [11]:

# Get page count without loading all pages
info = pdfinfo_from_path(PDF_PATH)
page_count = info["Pages"]

with open(OUTPUT_FILE, "w", encoding="utf-8") as out:
    for page_num in tqdm(range(1, page_count + 1)):
        # Convert one page at a time
        images = convert_from_path(
            PDF_PATH,
            dpi=200,
            first_page=page_num,
            last_page=page_num
        )
        img = images[0]
        img_path = f"../data/rigveda_images/page_{page_num}.png"
        img.save(img_path, "PNG")

        # Encode and send to Ollama
        # img_b64 = encode_image(img_path)
        response = chat(
            model=MODEL_NAME,
            # format=ImageDescription.model_json_schema(),  # Pass in the schema for the response
            messages=[
                {
                'role': 'user',
                'content': 'Analyze this image and return a detailed JSON description including objects, scene, colors and any text detected. If you cannot determine certain details, leave those fields empty.',
                'images': [img_path],
                },
            ],
            options={'temperature': 0},  # Set temperature to 0 for more deterministic output
            )
        

        # Write response to file
        out.write(f"--- Page {page_num} ---\n")
        out.write(f"{response.message.content}\n\n")

        # Clean up the temporary image
        os.remove(img_path)

print(f"All responses saved to {OUTPUT_FILE}")

  0%|          | 0/1853 [00:24<?, ?it/s]


KeyboardInterrupt: 